## Importing Libraries

In [1]:
import pandas as pd
import optuna

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool, cv
from lightgbm import LGBMRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, r2_score

## Configurations

In [2]:
# path of training dataset
data_path = 'data/processed/train_processed.csv'

FOLDS=5 # no. of folds for cross validation
SPLIT=0.2 # splitting size
SEED=42 # seed for reproducibility

In [3]:
# loading the training dataset
df = pd.read_csv(data_path)

In [4]:
# separating dependent and independent variables
X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

In [5]:
# splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=SPLIT, random_state=SEED)

## Model Training

In [6]:
# models for training
models = {
    'LinearRegression': LinearRegression(),
    'Lasso Regression': Lasso(),
    'Ridge Regression': Ridge(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'XGBRegressor': XGBRegressor(),
    'CatBoostRegressor': CatBoostRegressor(verbose=False),
    'LGBMRegressor': LGBMRegressor(verbose=-1),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'Support Vector Regressor': SVR(),
    
}

In [7]:
# comparing models
results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    results.append({
        'Model': name,
        'MSE': mean_squared_error(y_val, y_pred),
        'MAE': mean_absolute_error(y_val, y_pred),
        'RMSE': root_mean_squared_error(y_val, y_pred),
        'R2 Score': r2_score(y_val, y_pred)
    })

    results_df = pd.DataFrame(results).sort_values('R2 Score', ascending=False)

In [8]:
results_df

,Model,MSE,MAE,RMSE,R2 Score
7,CatBoostRegressor,0.044521,0.158761,0.210999,0.649946
6,XGBRegressor,0.045057,0.159137,0.212266,0.645731
8,LGBMRegressor,0.045186,0.160274,0.212571,0.644712
4,RandomForestRegressor,0.047826,0.162961,0.218692,0.623955
5,GradientBoostingRegressor,0.054454,0.179304,0.233354,0.571841
9,KNeighborsRegressor,0.064928,0.192371,0.254810,0.489487
10,Support Vector Regressor,0.074108,0.208277,0.272228,0.417308
3,DecisionTreeRegressor,0.078355,0.207861,0.279920,0.383911
2,Ridge Regression,0.088286,0.234646,0.297130,0.305830
0,LinearRegression,0.088287,0.234644,0.297132,0.305818


## Tuning Hyper-Parameters using Optuna

### 1. CatBoostRegressor

In [13]:
# objective function for tuning CatBoostRegressor
def objective(trail):
    params = {
        'iterations': trail.suggest_int('iterations', 500, 1500),
        'learning_rate': trail.suggest_float('learning_rate', 0.01, 0.1),
        'depth': trail.suggest_int('depth', 1, 10),
        'l2_leaf_reg': trail.suggest_int('l2_leaf_reg', 1, 10),
        'border_count': trail.suggest_int('border_count',10, 150),
        'random_strength': trail.suggest_float('random_strength', 0.1, 1.0),
        'bagging_temperature': trail.suggest_float('bagging_temperature', 0, 1.0),

        'loss_function': 'RMSE',
        'eval_metric': 'R2',
        'random_seed': SEED,
        'verbose': False
    }

    cv_results = cv(
        pool=Pool(X_train, y_train),
        params=params,
        fold_count=FOLDS,
        shuffle=True,
        stratified=False,
        verbose=False
    )

    best_r2 = max(cv_results['test-R2-mean'])

    return best_r2

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f'Best Parameters: {study.best_params}')

[I 2026-02-14 19:11:36,415] A new study created in memory with name: no-name-ac7f52f2-0d62-4516-8967-30e9a52cc7ee


Training on fold [0/5]

bestTest = 0.6349491216
bestIteration = 1364

Training on fold [1/5]

bestTest = 0.6269462284
bestIteration = 1364

Training on fold [2/5]

bestTest = 0.6308059756
bestIteration = 1364

Training on fold [3/5]

bestTest = 0.6314658907
bestIteration = 1362

Training on fold [4/5]


[I 2026-02-14 19:11:54,108] Trial 0 finished with value: 0.6306903474315056 and parameters: {'iterations': 1365, 'learning_rate': 0.04545321069179818, 'depth': 3, 'l2_leaf_reg': 3, 'border_count': 101, 'random_strength': 0.35413665127826044, 'bagging_temperature': 0.9064442446934077}. Best is trial 0 with value: 0.6306903474315056.



bestTest = 0.6293122791
bestIteration = 1363

Training on fold [0/5]

bestTest = 0.3927428083
bestIteration = 606

Training on fold [1/5]

bestTest = 0.397548966
bestIteration = 606

Training on fold [2/5]

bestTest = 0.393170206
bestIteration = 606

Training on fold [3/5]

bestTest = 0.3999037335
bestIteration = 606

Training on fold [4/5]


[I 2026-02-14 19:12:00,543] Trial 1 finished with value: 0.3957049375107343 and parameters: {'iterations': 607, 'learning_rate': 0.03758490624037313, 'depth': 1, 'l2_leaf_reg': 1, 'border_count': 112, 'random_strength': 0.32086249314172083, 'bagging_temperature': 0.6526851739720636}. Best is trial 0 with value: 0.6306903474315056.


...


bestTest = 0.6295818849
bestIteration = 1397

Training on fold [0/5]

bestTest = 0.6686072891
bestIteration = 1193

Training on fold [1/5]

bestTest = 0.652739669
bestIteration = 1169

Training on fold [2/5]

bestTest = 0.6628912971
bestIteration = 1213

Training on fold [3/5]

bestTest = 0.6594659097
bestIteration = 1239

Training on fold [4/5]


[I 2026-02-14 19:34:04,071] Trial 48 finished with value: 0.6599681164376175 and parameters: {'iterations': 1243, 'learning_rate': 0.09999100412807026, 'depth': 7, 'l2_leaf_reg': 5, 'border_count': 123, 'random_strength': 0.7662050201728497, 'bagging_temperature': 0.10607019813500934}. Best is trial 43 with value: 0.6636717394267887.



bestTest = 0.6571424974
bestIteration = 780

Training on fold [0/5]

bestTest = 0.6698890952
bestIteration = 1122

Training on fold [1/5]

bestTest = 0.6557355416
bestIteration = 1358

Training on fold [2/5]

bestTest = 0.6649958232
bestIteration = 1014

Training on fold [3/5]

bestTest = 0.6581689965
bestIteration = 1204

Training on fold [4/5]


[I 2026-02-14 19:34:36,557] Trial 49 finished with value: 0.6612281408973046 and parameters: {'iterations': 1440, 'learning_rate': 0.09298784720422713, 'depth': 7, 'l2_leaf_reg': 3, 'border_count': 140, 'random_strength': 0.2324883055329317, 'bagging_temperature': 0.8346897095108177}. Best is trial 43 with value: 0.6636717394267887.



bestTest = 0.6582229982
bestIteration = 1101

Best Parameters: {'iterations': 1485, 'learning_rate': 0.09566208338756467, 'depth': 8, 'l2_leaf_reg': 4, 'border_count': 149, 'random_strength': 0.3221392635228121, 'bagging_temperature': 0.16171642890213989}


In [ ]:
# validating the tuned CatBoostRegressor
best_params = study.best_params

tuned_CBR = CatBoostRegressor(**best_params)
tuned_CBR.fit(X_train, y_train)
y_pred = tuned_CBR.predict(X_val)

catboost_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

catboost_results_df = pd.DataFrame([catboost_results])

In [21]:
catboost_results_df

,MSE,MAE,RMSE,R2 Score
0,0.045235,0.159138,0.212686,0.644328


### 2. XGBRegressor

In [23]:
# objective function for tuning XGBRegressor
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample',0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 2.5),
        'random_state': SEED
    }

    model = XGBRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=FOLDS, scoring='r2').mean()

    return score

In [24]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f'Best Parameters: {study.best_params}')

[I 2026-02-14 19:37:20,856] A new study created in memory with name: no-name-bbdd3908-1049-4acb-be98-8b827e190b67
C:\Users\ansar\Desktop\Workspace\Personal\MLOPs\House Price Prediction\venv\Lib\site-packages\xgboost\training.py:200: UserWarning: [19:37:20] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "random_seed" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\ansar\Desktop\Workspace\Personal\MLOPs\House Price Prediction\venv\Lib\site-packages\xgboost\training.py:200: UserWarning: [19:37:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "random_seed" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\ansar\Desktop\Workspace\Personal\MLOPs\House Price Prediction\venv\Lib\site-packages\xgboost\training.py:200: UserWarning: [19:37:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:782: 
Parameters: { "random_seed" } are not used.

  bst.update(dtrain, ite

Best Parameters: {'n_estimators': 535, 'learning_rate': 0.057582817878155085, 'max_depth': 10, 'min_child_weight': 10, 'subsample': 0.7124624923071757, 'colsample_bytree': 0.7561466714445289, 'gamma': 0.0433184458126587, 'reg_alpha': 0.934442472471591, 'reg_lambda': 2.07724971668887}


In [25]:
# validating the tuned XGBRegressor
best_params = study.best_params

tuned_XGB = XGBRegressor(**best_params)
tuned_XGB.fit(X_train, y_train)
y_pred = tuned_XGB.predict(X_val)

xgb_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

xgb_results_df = pd.DataFrame([xgb_results])

In [26]:
xgb_results_df

,MSE,MAE,RMSE,R2 Score
0,0.043301,0.155576,0.208089,0.659535


### 3. LGBMRegressor

In [8]:
# objective function for LGBMRegressor

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'num_leaves': trial.suggest_int('num_leaves', 31, 200),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 200),
        'subsample': trial.suggest_float('subsample', 0.8, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.8, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1.0),
        'random_state': SEED,
        'verbose':-1
    }

    model = LGBMRegressor(**params)
    score = cross_val_score(model, X_train, y_train, cv=FOLDS, scoring='r2').mean()

    return score

In [9]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print(f'Best Parameters: {study.best_params}')

[I 2026-02-14 20:07:05,525] A new study created in memory with name: no-name-ffde3516-3541-4fcd-85f3-55b7b241bd54
[I 2026-02-14 20:07:10,823] Trial 0 finished with value: 0.6671900591722856 and parameters: {'n_estimators': 921, 'learning_rate': 0.08289229517955117, 'max_depth': 7, 'num_leaves': 185, 'min_child_samples': 29, 'subsample': 0.9071106279880145, 'colsample_bytree': 0.8262395832621486, 'reg_alpha': 0.6294709479551285, 'reg_lambda': 0.8995915270683216}. Best is trial 0 with value: 0.6671900591722856.
[I 2026-02-14 20:07:12,626] Trial 1 finished with value: 0.6608516078554858 and parameters: {'n_estimators': 616, 'learning_rate': 0.0617964460707792, 'max_depth': 5, 'num_leaves': 98, 'min_child_samples': 133, 'subsample': 0.9408333994198707, 'colsample_bytree': 0.851449526148705, 'reg_alpha': 0.5966186211772637, 'reg_lambda': 0.18656031323775435}. Best is trial 0 with value: 0.6671900591722856.
[I 2026-02-14 20:07:14,861] Trial 2 finished with value: 0.6644826147368906 and param

Best Parameters: {'n_estimators': 214, 'learning_rate': 0.10979784611692994, 'max_depth': 9, 'num_leaves': 75, 'min_child_samples': 33, 'subsample': 0.939496220559003, 'colsample_bytree': 0.8775803119044167, 'reg_alpha': 0.8615502636228614, 'reg_lambda': 0.8363793385956172}


In [10]:
# validating the tuned LGBMRegressor
best_params = study.best_params

tuned_LGBM = LGBMRegressor(**best_params)
tuned_LGBM.fit(X_train, y_train)
y_pred = tuned_LGBM.predict(X_val)

lgbm_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

lgbm_results_df = pd.DataFrame([lgbm_results])

In [11]:
lgbm_results_df

,MSE,MAE,RMSE,R2 Score
0,0.043555,0.155852,0.208699,0.657535


### 4. Ensemble of tuned models

In [9]:
# ensemble of tuned models
ensemble = StackingRegressor(
    estimators=[
        ('catboost', tuned_CBR),
        ('lgbm', tuned_LGBM),
        ('xgb', tuned_XGB)
], final_estimator=Ridge(), cv=FOLDS)

In [10]:
# training the ensemble on training dataset
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_val)

ensemble_results = {
    'MSE': mean_squared_error(y_val, y_pred),
    'MAE': mean_absolute_error(y_val, y_pred),
    'RMSE': root_mean_squared_error(y_val, y_pred),
    'R2 Score': r2_score(y_val, y_pred)
}

ensemble_results_df = pd.DataFrame([ensemble_results])

0:	learn: 0.3427820	total: 151ms	remaining: 3m 44s
1:	learn: 0.3303523	total: 158ms	remaining: 1m 56s
2:	learn: 0.3196038	total: 163ms	remaining: 1m 20s
3:	learn: 0.3105680	total: 168ms	remaining: 1m 2s
4:	learn: 0.3023515	total: 174ms	remaining: 51.4s
5:	learn: 0.2950244	total: 179ms	remaining: 44.1s
6:	learn: 0.2883986	total: 184ms	remaining: 38.8s
7:	learn: 0.2833888	total: 189ms	remaining: 34.9s
8:	learn: 0.2785328	total: 194ms	remaining: 31.8s
9:	learn: 0.2745195	total: 199ms	remaining: 29.3s
10:	learn: 0.2707785	total: 204ms	remaining: 27.3s
11:	learn: 0.2672414	total: 209ms	remaining: 25.6s
12:	learn: 0.2642183	total: 214ms	remaining: 24.2s
13:	learn: 0.2619823	total: 219ms	remaining: 23s
14:	learn: 0.2594563	total: 224ms	remaining: 21.9s
15:	learn: 0.2574520	total: 229ms	remaining: 21s
16:	learn: 0.2557050	total: 233ms	remaining: 20.1s
17:	learn: 0.2540944	total: 238ms	remaining: 19.4s
18:	learn: 0.2521885	total: 244ms	remaining: 18.8s
19:	learn: 0.2504778	total: 249ms	remainin

In [11]:
ensemble_results_df

,MSE,MAE,RMSE,R2 Score
0,0.04313,0.155094,0.207677,0.660881


The ensemble model combining CatBoost, XGBoost, and LightGBM achieved the best overall performance, with an MSE of 0.04313, MAE of 0.15509, RMSE of 0.20768, and an R² score of 0.66088.
